In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

#Import Dataset

In [3]:
df = pd.read_csv('course_clean.csv')
df

,userId,courseId,course_name,rating
0,21461,398,pap,5
1,31909,340,machine-learning-projects,5
2,36819,588,wind-energy,4
3,10995,372,music-business-foundations,5
4,22336,209,forensic-science,5
...,...,...,...,...
49995,35239,231,getting-started-with-essay-writing,5
49996,9936,186,excel-vba-for-creative-problem-solving-part-1,5
49997,10116,439,python,5
49998,7039,513,sustainability,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userId       50000 non-null  int64 
 1   courseId     50000 non-null  int64 
 2   course_name  50000 non-null  object
 3   rating       50000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [5]:
df.shape

(50000, 4)

In [6]:
df.nunique()

userId         40602
courseId         595
course_name      595
rating             5
dtype: int64

#Splitting Data

In [7]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import NMF
from surprise import SlopeOne
from surprise import SVD
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

#Pemodelan

In [8]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'courseId', 'rating']], reader)

In [9]:
"Uji 7"
benchmark = []
# Iterate over all algorithms

algorithms = [NMF(), SlopeOne()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=True)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.NMF object at 0x00000213673E50A0>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x00000213673E5070>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x00000213673E50A0>
Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7727  0.7926  0.7729  0.7766  0.7725  0.7774  0.0077  
Fit time          2.61    6.63    4.14    3.02    3.26    3.93    1.44    
Test time         0.06    0.52    0.08    0.07    0.07    0.16    0.18    
Done:  <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x00000213673E50A0> 


Starting:  <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x00000213673E5070>


C:\Users\User\AppData\Local\Temp\ipykernel_11156\3274272681.py:17: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7415  0.7458  0.7179  0.7426  0.7330  0.7362  0.0101  
Fit time          0.37    0.36    0.39    0.35    0.43    0.38    0.03    
Test time         0.11    0.09    0.10    0.13    0.09    0.10    0.02    
Done:  <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x00000213673E5070> 



	DONE



C:\Users\User\AppData\Local\Temp\ipykernel_11156\3274272681.py:17: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


In [10]:
results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
results

,test_rmse,fit_time,test_time
Algorithm,,,
SlopeOne,0.736167,0.381998,0.104145
NMF,0.777445,3.932326,0.160968
